In [201]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from wordcloud import WordCloud
from wordcloud import ImageColorGenerator
from wordcloud import STOPWORDS

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix
from sklearn.decomposition import PCA

In [202]:
from sklearn.decomposition import NMF
from rake_nltk import Rake

# df = pd.read_csv("English_fiction_pre_PCA_3.csv")
# df = df.iloc[:300]
# train_index = list(range(0, 241))
# test_index = list(range(241, 301))

df = pd.read_csv("English_fiction_pre_PCA_3.csv")
train_df = pd.read_csv("original_data/train_indices.csv")
test_df = pd.read_csv("original_data/test_indices.csv")
train_index = train_df["index"].tolist()
test_index = test_df["index"].tolist()

In [203]:
def concat_nouns(text):
    """
    concatenate and apply lowercase lettering to proper nouns like names, publishers, and book titles
    """

    # Step 1: remove leading or ending brackets (if applicable) and internal quote marks
    text = text.strip("[]")
    text = text.replace("'", "")

    # Step 2: If there are multiple nouns, split at the comma
    text = text.split(", ")

    # Step 3: Concatenate each noun and put all letters in lowercase:
    text = [x.replace(" ", "").lower() for x in text]

    # Step 4: Convert the list of tokens to a string
    text = ' '.join(text)

    return text

def add_tokens_to_description(df):
    df["description"] += " " + df["authors"].apply(concat_nouns)
    df["description"] += " " + df["publisher"].apply(concat_nouns)
    df["description"] += " " + df["Title"].str.lower()

def calculate_ngrams_RAKE(text: str):
    r_unigram = Rake()
    r_unigram.extract_keywords_from_text(text)
    
    keyword_dict_scores = r_unigram.get_word_degrees()
    words = list(keyword_dict_scores.keys())
    
    return " ".join(words)

def create_tokens(df, input_column: str, output_column: str):  
    df[output_column] = df["description"].apply(calculate_ngrams_RAKE)

# Calculate tfidf matrix:
def calculate_TFIDF(df, BOW_column: str, train_index, test_index):
    # Split the incoming dataframe into train and test slices base on the list of train and test indices provided:
    X_train_df = df[df["index"].isin(train_index)]
    X_test_df = df[df["index"].isin(test_index)]
    
    #instantiating and generating the tfidf
    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(X_train_df[BOW_column])
    X_test_tfidf = vectorizer.transform(X_test_df[BOW_column])

    # convert the tfidf matrix to a dense matrix
    dense_X_train_tfidf = X_train_tfidf.toarray()
    dense_X_test_tfidf = X_test_tfidf.toarray()

    # # combine the arrays:
    # dense_X = np.concatenate((dense_X_train_tfidf, dense_X_test_tfidf), axis = 0)

    # Determine the column names for our dense matrix and create a dataframe with the 
    # vocabulary as columns:
    temp_dict = {}
    for counter, i in enumerate(list(vectorizer.vocabulary_.items())):
            temp_dict[i[1]] = i[0]
    
    column_names = []
    for i in range(len(temp_dict)):
        column_names.append(temp_dict[i])

    # Convert the array back into a dataframe:
    scaled_dataframe_X_train=pd.DataFrame(dense_X_train_tfidf, columns= column_names)
    scaled_dataframe_X_test=pd.DataFrame(dense_X_test_tfidf, columns= column_names) 

    return scaled_dataframe_X_train, scaled_dataframe_X_test, column_names

In [204]:
# Add tokens from other columns to the description column, specifically author, title, and publisher
add_tokens_to_description(df)

# Create tokens from the book descriptions and save this in a new column called "tokens"
create_tokens(df, "description", "tokens")

X_train_tfidf, X_test_tfidf, column_names = calculate_TFIDF(df, "tokens", train_index, test_index)

In [205]:
X_train_tfidf.shape

(21419, 80652)

In [206]:
NMF_model = NMF(n_components=100)

# Fit the model to the training tfidf matrix
NMF_model.fit(X_train_tfidf)

# Transform the articles: nmf_features
nmf_features = NMF_model.transform(X_train_tfidf)

# Print the NMF features
print(nmf_features.round(2))

[[0.   0.   0.   ... 0.   0.   0.03]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.02 0.05 0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]]


In [207]:
nmf_features_test = NMF_model.transform(X_test_tfidf)

In [209]:
print(len(train_index))
print(nmf_features.shape)

print(len(test_index))
print(nmf_features_test.shape)

21419
(21419, 100)
5355
(5355, 100)


In [210]:
components_df = pd.DataFrame(NMF_model.components_, columns=column_names)

In [213]:
# Print the shape of the DataFrame
print(components_df.shape)

# Select row 3: component
component = components_df.iloc[50]

# Print result of nlargest
print(component.nlargest(20))

(100, 80652)
friend        4.547039
best          4.352509
selling       0.697111
friendship    0.347126
dog           0.247043
away          0.210772
childhood     0.157979
ever          0.141236
loved         0.137807
summer        0.134270
helps         0.125767
close         0.123370
also          0.118257
full          0.114116
seller        0.110791
takes         0.109695
jason         0.107355
better        0.107301
college       0.106310
decide        0.099847
Name: 50, dtype: float64


In [181]:
topics = {0: "nostalgia", 
          1: "self-published/debut",
          2: "story/anthology",
          3: "womens_fiction",
          4: "childrens_books",
          5: "classic",
          6: "family_drama",
          7: "digital_books/recreations",
          8: "reproduced",
          9: "murder_mystery",
          10: "reprint",
          11: "bestselling_author",
          12: "romance",
          13: "unkonwn",
          14: "teen",
          15: "novel",
          16: "world/war/historical_fiction",
          17: "unknown",
          18: "young_adult",
          19: "coming_of_age",
         }

In [200]:
X_train_df = df[df["index"].isin(train_index)]
X_test_df = df[df["index"].isin(test_index)]
book_index = 12

# retrieve the nmf features for a predefined index
nmf_feature_list = nmf_features_test[book_index].round(2).tolist()
print(nmf_feature_list)

# calculate the maxium nmf value(s)
max_nmf_value = max(nmf_feature_list)

# find the indices in the nmf feature list to be used to convert the values to category labels
indices = [i for i, x in enumerate(nmf_feature_list) if x == max_nmf_value]
indices

category_label = [topics[i] for i in indices]
print(category_label)
X_test_df["description"].iloc[book_index]

[0.02, 0.0, 0.0, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02]
['romance']


"Always The Chaperon And Never The Bride... At least, that's the way it was for Lady Annis Wyncherley. If this young widow was to remain as chaperon to society's misses, there could be no hint of scandal attached to her name. Rakes and romance were strictly off-limits, most especially a rogue like the handsome Lord Adam Ashwick! But that proved nearly impossible when Adam made his daughter's chaperon the subject of his relentless seduction. Adam knew any attention from him could destroy Lady Wyncherley's fine reputation. But he was powerless to control the strong desires she aroused in him. And all too soon this reformed rogue was hell-bent on convincing a very stubborn Annis to become his chaperon bride.... nicolacornick harlequin chaperon bride, the (historical romance)"

In [214]:
for i in range(nmf_features.shape[1]):
    location= X_train_df.shape[1]
    X_train_df.insert(location, f"NMF_{i}", nmf_features[:,i].tolist())

for i in range(nmf_features_test.shape[1]):
    location= X_test_df.shape[1]
    X_test_df.insert(location, f"NMF_{i}", nmf_features_test[:,i].tolist())

/var/folders/62/rv9lbyxx6l110tffps2wcg2r0000gn/T/ipykernel_15362/1199177461.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train_df.insert(location, f"NMF_{i}", nmf_features[:,i].tolist())
/var/folders/62/rv9lbyxx6l110tffps2wcg2r0000gn/T/ipykernel_15362/1199177461.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train_df.insert(location, f"NMF_{i}", nmf_features[:,i].tolist())
/var/folders/62/rv9lbyxx6l110tffps2wcg2r0000gn/T/ipykernel_15362/1199177461.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is

In [183]:
nmf_features_test.shape

(5355, 20)

In [188]:
nmf_features.shape[1]

20

In [215]:
print(X_train_df.shape)
print(X_test_df.shape)

(21419, 119)
(5355, 119)


In [190]:
topics[0]

'nostalgia'

In [217]:
X_test_df.head(2)

,Title,description,authors,image,previewLink,publisher,infoLink,categories,index,reviews number,...,NMF_90,NMF_91,NMF_92,NMF_93,NMF_94,NMF_95,NMF_96,NMF_97,NMF_98,NMF_99
6,From Potter's Field,"The sixth book in the Kay Scarpetta series, fr...",['Patricia Cornwell'],http://books.google.com/books/content?id=prefg...,http://books.google.nl/books?id=prefgSxnGOwC&p...,Hachette UK,https://play.google.com/store/books/details?id...,['fiction'],115,157,...,0.000625,0.000343,0.0,0.0,0.002655,0.003043,0.0,0.00958,0.018879,0.000346
17,Riverworld and Other Stories,Three stories of a world shared by resurrected...,['Philip José Farmer'],http://books.google.com/books/content?id=TP4oD...,http://books.google.nl/books?id=TP4oDwAAQBAJ&p...,Open Road Media,https://play.google.com/store/books/details?id...,['fiction'],209,7,...,0.000978,0.019001,0.0,0.0,0.000000,0.021188,0.0,0.00000,0.000852,0.013162


In [218]:
X_train_df.to_csv(
path_or_buf = "X_train_NMF_topics_100.csv",
index = False
)

X_test_df.to_csv(
path_or_buf = "X_test_NMF_topics_100.csv",
index = False
)